<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/jjc/NBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
datas = []
data = ['me free lottery', 'spam']
datas.append(data)
data = ['free get free you', 'spam']
datas.append(data)
data = ['you free scholarship', 'normal']
datas.append(data)
data = ['free to contact me', 'normal']
datas.append(data)
data = ['you won award', 'normal']
datas.append(data)
data = ['you ticket lottery', 'spam']
datas.append(data)

datas

[['me free lottery', 'spam'],
 ['free get free you', 'spam'],
 ['you free scholarship', 'normal'],
 ['free to contact me', 'normal'],
 ['you won award', 'normal'],
 ['you ticket lottery', 'spam']]

In [ ]:
datas = []
data = ['I love you', '긍정']
datas.append(data)
data = ['love happy weekend', '긍정']
datas.append(data)
data = ['bore work job', '부정']
datas.append(data)
data = ['I hate you', '중립']
datas.append(data)
data = ['bore weekend', '부정']
datas.append(data)
data = ['happy together', '긍정']
datas.append(data)

datas

[['I love you', '긍정'],
 ['love happy weekend', '긍정'],
 ['bore work job', '부정'],
 ['I hate you', '중립'],
 ['bore weekend', '부정'],
 ['happy together', '긍정']]

In [ ]:
import pandas as pd
import numpy as np
import itertools
import nltk

# Input 데이터프레임은 첫 번째 column에 words(split처리 된 list), 두 번째 column에 target
# Output 데이터프레임은 인덱스에 단어, 칼럼에 타겟
class NBC():
    
    df = pd.DataFrame([])
    k = 0.5

    def __init__(self, **kwargs):
        try:
            self.k = kwargs['k']
        except:
            pass
            
    def add_data(self, datas):
        word_column = datas.columns[0]
        target_column = datas.columns[1]
        target_ls = list(set(datas[word_column]))
        words = {}

        total_ngram = list(itertools.chain(*list(datas[word_column]))) # list로 들어가지 않을 경우
        unique_ngram = list(set(total_ngram))

        result_df = pd.DataFrame(unique_ngram, columns = [word_column]).set_index(word_column)

        for target in target_ls:
            this_ngram = list(itertools.chain(*list(datas[datas[target_column] == target][word_column])))
            fdist = nltk.FreqDist(this_ngram)
            temp_df = pd.DataFrame(list(zip(fdist.keys(), fdist.values())), columns= [word_column, 'count']).set_index(word_column)
            result_df[target] = temp_df['count']

        result_df.fillna(0)
        self.df = result_df
        return self.df

    def pol_score(self, target, s):
        df = self.df
        k = self.k

        totals = {}
        for i in df:
            totals[i] = df[i].sum()
        
        tokens = s.split()
        total = np.asarray(list(totals.values())).sum()

        logs = {}
        for i in df:            
            logs[i] = np.log(float(totals[i]) / float(total))
        
        for token in tokens:
            for t in df:
                logs[t] += np.log( (k + df[t][token]) / (2*k +totals[t]) )

        bunmo = 0
        for log in logs:
            bunmo += np.exp(logs[log])
        bunja = np.exp(logs[target])
        return bunja / bunmo

In [ ]:
import pandas as pd
test = pd.DataFrame(datas, columns = ['words', 'target'])

nbc = NBC()
nbc.add_data(test)

,부정,긍정,중립
words,,,
work,1,0,0
you,0,1,1
happy,0,2,0
bore,2,0,0
love,0,2,0
together,0,1,0
weekend,1,1,0
job,1,0,0
hate,0,0,1


In [ ]:
nbc.pol_score('긍정', 'happy weekend')

0.7103218645948945